# Late Fusion with Weighted Average

Programmed By: Rabin Nepal

Supervisor Name: Dr. Bonny Banerjee

University of Memphis

Date: Dec, 2023

## References 

* Fusion Algorithm: https://github.com/cerlymarco/MEDIUM_NoteBook/blob/master/NeuralNet_Ensemble/NeuralNet_Ensemble.ipynb
* Dataset: https://www.eecs.qmul.ac.uk/mmv/datasets/deap/doc/tac_special_issue_2011.pdf
* SOTA: https://paperswithcode.com/sota/eeg-emotion-recognition-on-deap


## How to use the code?

change directory to codes and make a virtual environment for python3.8
* conda create -n deap_venv python=3.8

activate the new virtial environment
* conda activate deap_venv

install the required python packages 
* pip3 install -r requirements.txt


In [1]:
# import necessary modules
import os
import re
import tqdm
import glob
import pandas as pd
import numpy as np
import itertools
import math

from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Layer, LeakyReLU
from tensorflow.keras import Model
from tensorflow.keras.initializers import glorot_uniform,glorot_normal,he_normal,he_uniform
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# Set Keras backend seed (TensorFlow backend) for uniform training output
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve,classification_report

In [ ]:
np

In [ ]:

# global definitions
MAIN_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(MAIN_DIR, "preprocessed_datasets/DEAP")
FEATURE_SET = ["feat_set1","feat_set2","feat_set3"]

DATA_DIR =  os.path.join(DATA_DIR,FEATURE_SET[0])

SUBJECTS = 32 # 40 samples per subject
TEST_SUBJECT = 1 

# VALANCE Labels: 1-5 is unpleasant and 5-9 is pleasant
V_LABELS = ["Unpleasant","Pleasant"]

# Arousal Labels: 1-5 is Calm and 5-9 is Excited
A_LABELS = ["Calm","Excited"]

NUM_MODALITIES = 7
MODALITIES = ["EEG", "EOG", "EMG", "GSR", "RESP", "plet", "temp"]
MOD_KEYS = ['mod1', 'mod2', 'mod3', 'mod4', 'mod5', 'mod6', 'mod7']

# create a dictionary where each modality is mapped to its respective index, starting from 1
MODALITY_INDICES = {modality: index + 1 for index, modality in enumerate(MODALITIES)}

In [ ]:
# get files from the dataset directory
files = glob.glob(DATA_DIR+"/*.mat")
files.sort()

In [ ]:
files

In [ ]:
sub_info = loadmat(files[-1])[list(loadmat(files[-1]).keys())[-1]]
sub_info

In [ ]:
sample_data = loadmat(files[3])
sample_data

In [ ]:
def extract_data(files):
    """
    Extracts data from the provided list of files.

    Parameters:
    - files (list): A list containing file paths to extract data from.

    Returns:
    - data (dict): A dictionary containing data extracted from the files categorized by MOD_KEYS.
    - labels_a (list): List of 'lab_a' files' content.
    - labels_v (list): List of 'lab_v' files' content.

    The function iterates through the provided list of files to extract data based on the provided MOD_KEYS.
    Additionally, it separates 'lab_a' and 'lab_v' files' content into separate lists: labels_a and labels_v.
    """
    
    data = {key: [] for key in MOD_KEYS}
    
    # Define a regex pattern to extract the modality number
    pattern = r'DEAP_mod(\d+)_data.mat'

    labels_a = []
    labels_v = []
    
    def read_file(file):
        d = loadmat(file)
        d_key = list(d.keys())[-1]
        return d[d_key]
    
    for file in tqdm.tqdm(files,desc="Extracting Data ..."):
        if(("lab" in file) or ("sub" in file)):
            if("lab_a" in file):
                labels_a.append(read_file(file))                
            
            if("lab_v" in file):
                labels_v.append(read_file(file))                
            continue
        
        # Extract modality number from file names using regex
        key = int(re.search(pattern, file).group(1)) - 1
        # print(key)
        data[MOD_KEYS[key]] = read_file(file)
    
    return data,labels_a[0],labels_v[0]

In [ ]:
data , labels_a , labels_v = extract_data(files=files)

In [ ]:
for count,mod in enumerate(MODALITIES):
    print(f"Shape of Modality-{count+1} ({MODALITIES[count]}) :{data[MOD_KEYS[count]].shape}")

In [ ]:
print(labels_v.shape)
print(labels_a.shape)

In [ ]:
# print(labels_a[:5])
print(labels_v[:5])

## Definition for Model Evaluation

In [ ]:
class Metrics:
    """
    A class to compute and visualize performance metrics for the trained model.

    Attributes:
    - model: The trained machine learning model.
    - history: Training history of the model.
    - X_test: Test data used for evaluation.
    - y_test: True labels of the test data.

    Methods:
    - plot_loss_accuracy(): Plot training and validation accuracy & loss over epochs.
    - plot_confusion_matrix(): Plot the confusion matrix.
    - calculate_precision_recall(): Calculate precision-recall curve values.
    - calculate_precision_recall_f1(): Calculate precision, recall, and F1-score.
    - calculate_auc(): Calculate the Area Under the ROC Curve (AUC).
    - plot_roc_curve(): Plot the Receiver Operating Characteristic (ROC) curve.
    """
    
    def __init__(self, model, history, X_test, y_test):
        self.model = model
        self.history = history
        self.X_test = X_test
        self.y_test = np.array([np.argmax(y) for y in y_test])
        self.y_pred = np.argmax(model.predict(X_test),axis=1) 
        
        self.labels = [0,1]

    def plot_loss_accuracy(self):
        plt.figure(figsize=(12, 4))
        # Plot training & validation accuracy values
        plt.subplot(1, 2, 1)
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(['Train', 'Test'], loc='upper left')

        # Plot training & validation loss values
        plt.subplot(1, 2, 2)
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('Model loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Train', 'Test'], loc='upper left')

        plt.tight_layout()
        plt.show()

    def plot_confusion_matrix(self):
        cm = confusion_matrix(self.y_test, self.y_pred, labels=self.labels)
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.colorbar()

        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j,i, format(cm[i,j],"d"),ha="center", va="center",color="white" if cm[i, j] > thresh else "black")
                
        tick_marks = np.arange(len(self.labels))
        plt.xticks(tick_marks, self.labels)
        plt.yticks(tick_marks, self.labels)
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.tight_layout()
        plt.show()

    def calculate_precision_recall(self):
        precision, recall, _ = precision_recall_curve(self.y_test, self.y_pred)
        return precision, recall
    

    def calculate_precision_recall_f1(self):
        return classification_report(y_true=self.y_test,y_pred=self.y_pred)

    def calculate_auc(self):
        fpr, tpr, _ = roc_curve(self.y_test, self.y_pred)
        roc_auc = auc(fpr, tpr)
        return fpr, tpr, roc_auc

    def plot_roc_curve(self):
        fpr, tpr, roc_auc = self.calculate_auc()
        plt.figure(figsize=(6, 4))
        plt.plot(fpr, tpr, lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.show()


## Definition of Modality Class to Handle Multi-Modality

In [ ]:
class Modality:
    """
    A class for handling modality data, including building, training, and evaluating a neural network model.

    Args:
    - data (numpy.ndarray): Input data for the each modality.
    - labels (numpy.ndarray): Corresponding labels for the input data.
    - regularization (float): Regularization rate for dropout layers, if applied.

    Methods:
    - flatten(data): Method to flatten the input data.
    - train_test_split(test_size=40, random_state=42): Splits the data into training and test sets.
    - build_model(neurons): Builds a neural network model with specified number of neurons.
    - train_model(model_name, epochs=1000): Trains the built model for a specified number of epochs.
    - plots(): Generates and displays plots for model evaluation metrics.
    - get_history(): Retrieves the training history of the model.
    - get_model(model_name): Retrieves a trained model given the model's name.
    - build_layers(model_name): Builds layers for a trained model to analyze weights and biases.

    Attributes:
    - labels (numpy.ndarray): Labels corresponding to the data.
    - data (numpy.ndarray): Input data for the modality.
    - regularization (float): Regularization rate for dropout layers, if applied.
    - X_train (numpy.ndarray): Training data after splitting.
    - X_test (numpy.ndarray): Test data after splitting.
    - y_train (numpy.ndarray): Training labels after splitting.
    - y_test (numpy.ndarray): Test labels after splitting.
    - model (tf.keras.Model): Neural network model.
    - trained_model (tf.keras.Model): Trained neural network model.
    - history (tf.keras.callbacks.History): Training history of the model.
    - hidden_nodes (int): Number of neurons in the hidden layer.

    Usage:
    Initialize the class with data and labels, then proceed to build, train, and evaluate the model.
    """
    def __init__(self, data, labels,regularization=None):
        self.labels = labels
        self.data = data
        self.regularization = regularization
        
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.trained_model = None
        self.history = None
        self.hidden_nodes = None
        
        self.data = self.flatten(self.data)
        self.train_test_split()

    @staticmethod
    def flatten(data):
        print("Shape of Original Data:", data.shape)
        data = data.reshape(len(data), -1)
        print("Shape of Reshaped Data:", data.shape)
        return data

    def train_test_split(self, test_size=40, random_state=42):
        # Split data into train and test sets
        X_train = self.data[:-test_size]
        X_test = self.data[-test_size:]
        y_train = self.labels[:-test_size]
        y_test = self.labels[-test_size:]

        # Shuffle indices
        num_samples = len(X_train)
        shuffled_indices = np.random.RandomState(seed=random_state).permutation(num_samples)

        # Use shuffled indices to shuffle both features and labels
        self.X_train = X_train[shuffled_indices]
        self.y_train = y_train[shuffled_indices]
        self.X_test = X_test
        self.y_test = y_test

        print("X_train shape:", self.X_train.shape)
        print("X_test shape:", self.X_test.shape)
        print("y_train shape:", self.y_train.shape)
        print("y_test shape:", self.y_test.shape)


    def build_model(self,neurons):
        self.hidden_nodes=neurons
        Inputs = Input(shape=(int(self.data[0].shape[0],),))
        x = Dense(units=neurons, activation="linear", kernel_initializer=he_normal(seed=42))(Inputs)
        x = LeakyReLU(alpha=0.4)(x)
        if(self.regularization is not None):
            x = Dropout(self.regularization)(x)
        Outputs = Dense(units=2, activation="softmax", kernel_initializer=he_normal(seed=42))(x)

        self.model = Model(Inputs, Outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
        self.model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        self.model.summary()


    def train_model(self, model_name, epochs=1000):
        model_name = model_name + '--{epoch:02d}--{val_accuracy:.2f}.h5'

        def save_best_model(save_as):
            checkpoint = ModelCheckpoint(filepath=save_as,
                                         monitor='val_accuracy',
                                         verbose=0,
                                         save_best_only=True,
                                         mode='max')
            return checkpoint

        history = self.model.fit(self.X_train, self.y_train,
                                 epochs=epochs,
                                 shuffle=False,
                                 batch_size=128,
                                 validation_data=(self.X_test, self.y_test),
                                 callbacks=[save_best_model(model_name)],
                                 verbose=0)

        self.history = history



    def plots(self):
        metrics = Metrics(self.model, self.history, self.X_test, self.y_test)
        metrics.plot_loss_accuracy()
      
        print(metrics.calculate_precision_recall_f1())
        metrics.plot_confusion_matrix()
        metrics.plot_roc_curve()


    def get_history(self):
        return self.history

    
    def get_model(self,model_name):
        return load_model(model_name)
    
    
    def build_layers(self,model_name):
        self.trained_model = load_model(model_name)

        kernel_initializer1 = tf.keras.initializers.constant(self.trained_model.trainable_weights[0])
        bias_initializer1 = tf.keras.initializers.constant(self.trained_model.trainable_weights[1])

        kernel_initializer2 = tf.keras.initializers.constant(self.trained_model.trainable_weights[2])
        bias_initializer2 = tf.keras.initializers.constant(self.trained_model.trainable_weights[3])

        
        Inputs = Input(shape=(int(self.data[0].shape[0],),))
        x = Dense(units=self.hidden_nodes, 
                  activation="linear", 
                  kernel_initializer = kernel_initializer1,
                  bias_initializer= bias_initializer1)(Inputs)
        
        x = LeakyReLU(alpha=0.4)(x)
        
        x = Dense(units=2, activation="softmax", 
                        kernel_initializer = kernel_initializer2, 
                        bias_initializer = bias_initializer2)(x)
        
        return Inputs,x


## Working with MODALITY 1 (EEG)

In [ ]:
modality_1 = Modality(data=data['mod1'],labels=labels_v,regularization=0.6)
modality_1.build_model(neurons=33)
modality_1.train_model(model_name="mod1",epochs=1000)
modality_1.plots()

## Working with MODALITY 2 (EOG)

In [ ]:
modality_2 = Modality(data=data['mod2'],labels=labels_v)
modality_2.build_model(neurons=12)
modality_2.train_model(model_name="mod2",epochs=2000)
modality_2.plots()

## Working with MODALITY 3 (EMG)

In [ ]:
modality_3 = Modality(data=data['mod3'],labels=labels_v)
modality_3.build_model(neurons=11)
modality_3.train_model(model_name="mod3",epochs=1500)
modality_3.plots()

## Working with MODALITY 4 (GSR)

In [ ]:
modality_4 = Modality(data=data['mod4'],labels=labels_v)
modality_4.build_model(neurons=9)
modality_4.train_model(model_name="mod4",epochs=2000)
modality_4.plots()

## Working with MODALITY 5 (RESP)

In [ ]:
modality_5 = Modality(data=data['mod3'],labels=labels_v)
modality_5.build_model(neurons=13)
modality_5.train_model(model_name="mod5",epochs=1500)
modality_5.plots()

## Working with MODALITY 6 (PLET)

In [ ]:
modality_6 = Modality(data=data['mod6'],labels=labels_v)
modality_6.build_model(neurons=9)
modality_6.train_model(model_name="mod6",epochs=1500)
modality_6.plots()

## Working with MODALITY 7 (TEMP)

In [ ]:
modality_7 = Modality(data=data['mod7'],labels=labels_v)
modality_7.build_model(neurons=8)
modality_7.train_model(model_name="mod7",epochs=3000)
modality_7.plots()

# Weighted Average Fusion Method

In [ ]:
total_models = 0
min_modalities = 1
max_modalities = 7

# Calculate the total number of models for combinations from 1 to 7 modalities
for k in range(min_modalities, max_modalities + 1):
    combinations = math.comb(max_modalities, k)
    total_models += combinations

print("Total number of models for different combinations of 1 to 7 modalities:", total_models)


In [ ]:
class WeightedAverage(Layer):

    def __init__(self, n_output):
        super(WeightedAverage, self).__init__()
        self.W = tf.Variable(initial_value=tf.random.uniform(shape=[1,1,n_output], minval=0, maxval=1,seed=42),trainable=True)

    def call(self, inputs):
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs)
        weights = tf.nn.softmax(self.W, axis=-1)
        return tf.reduce_sum(weights*inputs, axis=-1)

## ALL MODALITIES

In [ ]:
saved_model_files = glob.glob("*.h5")

modalities = {}

pattern = r'(mod\d+)--\d+--(\d+\.\d+)\.h5'

for file in saved_model_files:
    match = re.match(pattern, file)
    if match:
        modality, accuracy = match.groups()
        accuracy = float(accuracy)
        
        if modality not in modalities or accuracy > modalities[modality]['accuracy']:
            modalities[modality] = {'accuracy': accuracy, 'filename': file}

for modality, data in modalities.items():
    print(f"Max accuracy for {modality}: {data['accuracy']} (Filename: {data['filename']})")
    

In [ ]:
modalities["mod1"]["filename"]

### Load Trained Weights

In [ ]:
# get the saved best performing models in each modality

# MOD1_INPUT, mod1_mlp = modality_1.build_layers(model_name="mod1--726--0.43.h5")
# MOD2_INPUT, mod2_mlp = modality_2.build_layers(model_name="mod2--989--0.62.h5")
# MOD3_INPUT, mod3_mlp = modality_3.build_layers(model_name="mod3--44--0.70.h5")
# MOD4_INPUT, mod4_mlp = modality_4.build_layers(model_name="mod4--1428--0.60.h5")
# MOD5_INPUT, mod5_mlp = modality_5.build_layers(model_name="mod5--238--0.55.h5")
# MOD6_INPUT, mod6_mlp = modality_6.build_layers(model_name="mod6--687--0.60.h5")
# MOD7_INPUT, mod7_mlp = modality_7.build_layers(model_name="mod7--539--0.57.h5")


MOD1_INPUT, mod1_mlp = modality_1.build_layers(model_name= modalities["mod1"]["filename"])
MOD2_INPUT, mod2_mlp = modality_2.build_layers(model_name= modalities["mod2"]["filename"])
MOD3_INPUT, mod3_mlp = modality_3.build_layers(model_name= modalities["mod3"]["filename"])
MOD4_INPUT, mod4_mlp = modality_4.build_layers(model_name= modalities["mod4"]["filename"])
MOD5_INPUT, mod5_mlp = modality_5.build_layers(model_name= modalities["mod5"]["filename"])
MOD6_INPUT, mod6_mlp = modality_6.build_layers(model_name= modalities["mod6"]["filename"])
MOD7_INPUT, mod7_mlp = modality_7.build_layers(model_name= modalities["mod7"]["filename"])



In [ ]:
# fusion models and input list
all_models = [mod1_mlp, mod2_mlp, mod3_mlp, mod4_mlp, mod5_mlp, mod6_mlp, mod7_mlp]

all_inputs = [MOD1_INPUT,
              MOD2_INPUT,
              MOD3_INPUT,
              MOD4_INPUT,
              MOD5_INPUT,
              MOD6_INPUT,
              MOD7_INPUT]

all_train = [modality_1.X_train,
             modality_2.X_train,
             modality_3.X_train,
             modality_4.X_train,
             modality_5.X_train,
             modality_6.X_train,
             modality_7.X_train]

all_train_labels = modality_1.y_train
            
all_test = [modality_1.X_test,
            modality_2.X_test,
            modality_3.X_test,
            modality_4.X_test,
            modality_5.X_test,
            modality_6.X_test,
            modality_7.X_test]

all_test_labels = modality_1.y_test

In [ ]:
def build_fused_model(selected_models,selected_inputs):
    model = selected_models
    wt_avg_layer = WeightedAverage(n_output=len(model))(model)
    output =  Dense(2, activation="softmax", kernel_initializer=he_normal(seed=42))(wt_avg_layer)

    model = Model(inputs=selected_inputs,
                        outputs=output)

    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics="accuracy")
    # model.summary()
    return model

def train_fused_model(model,X_train,y_train,X_test,y_test):
    history = model.fit(X_train, y_train,
                        epochs=1000,
                        batch_size=128,
                        verbose=0,
                        shuffle=False, 
                        validation_data=(X_test,y_test))
    return model, history

In [ ]:
selected_train_labels = all_train_labels
selected_test_labels = all_test_labels
    
def train_combinations(num_modalites):
    all_accuracies = []
    all_mod_combinations = []
    combinations = list(itertools.combinations(range(1, 8), num_modalites))

    progress_bar = tqdm.tqdm(desc="Training different combinations", total=len(combinations), dynamic_ncols=True)
    for combination in combinations:
        mod_combinations = [MODALITIES[i - 1] for i in combination]
        # print(mod_combinations)
        # progress_bar.set_description(f"Training model with modalities: {combination} or {mod_combinations}", refresh=True)
        print(f"Training model with modalities: {combination} or {mod_combinations}")
        selected_models = [all_models[i - 1] for i in combination]
        selected_inputs = [all_inputs[i - 1] for i in combination]
        selected_train = [all_train[i - 1] for i in combination]
        selected_test = [all_test[i - 1] for i in combination]

        model = build_fused_model(selected_models, selected_inputs)
        _, history = train_fused_model(model,
                                       selected_train,
                                       selected_train_labels,
                                       selected_test,
                                       selected_test_labels)

        max_accuracy = np.max(history.history["val_accuracy"])
        print("Maximum Validation Accuracy for this combination: {:.2f}".format(max_accuracy))

        all_accuracies.append(max_accuracy)
        all_mod_combinations.append(mod_combinations)

        progress_bar.update(1)

    progress_bar.close()
    return all_accuracies, all_mod_combinations

In [ ]:
def evaluate_fused_model(best_combination):
    selected_models = [all_models[i-1] for i in best_combination]
    selected_inputs = [all_inputs[i-1] for i in best_combination]
    selected_train = [all_train[i-1] for i in best_combination]
    selected_test = [all_test[i-1] for i in best_combination]
    best_model = build_fused_model(selected_models, selected_inputs)
    best_model, history = train_fused_model(best_model,
                                        selected_train,
                                        selected_train_labels,
                                        selected_test,
                                        selected_test_labels)

    best_model_metrics = Metrics(best_model,history,selected_test, all_test_labels)
    best_model_metrics.plot_loss_accuracy()
    print(best_model_metrics.calculate_precision_recall_f1())
    best_model_metrics.plot_confusion_matrix()
    best_model_metrics.plot_roc_curve()
        

In [ ]:
# build the weighted average fusion layer
mod1_7 = all_models
wt_avg_layer = WeightedAverage(n_output=len(mod1_7))(mod1_7)
output =  Dense(2, activation="softmax")(wt_avg_layer)

mod1_7_model = Model(inputs=all_inputs,
                     outputs=output)

mod1_7_model.compile(loss="binary_crossentropy",optimizer="adam",metrics="accuracy")
mod1_7_model.summary()

In [ ]:
mod1_7_model_history = mod1_7_model.fit(all_train,
                                        all_train_labels ,
                                        epochs=1000,
                                        batch_size=128,
                                        verbose=2,
                                        shuffle=False, 
                                        validation_data=(all_test,
                                                         all_test_labels))

In [ ]:
# see the learned value of weights
tf.nn.softmax(mod1_7_model.get_weights()[-3]).numpy()

In [ ]:
mod1_7_metrics = Metrics(mod1_7_model,mod1_7_model_history,all_test, all_test_labels)
mod1_7_metrics.plot_loss_accuracy()
print(mod1_7_metrics.calculate_precision_recall_f1())
mod1_7_metrics.plot_confusion_matrix()
mod1_7_metrics.plot_roc_curve()

In [ ]:
all_mod_accuracies, all_mod_combinations = train_combinations(num_modalites=7)

In [ ]:
max_acc_index = np.argmax(all_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for seven modalities is {all_mod_accuracies[max_acc_index]:.2f} for {all_mod_combinations[max_acc_index]}")


## SIX MODALITIES

In [ ]:
six_mod_accuracies, six_mod_combinations = train_combinations(num_modalites=6)

In [ ]:
max_acc_index = np.argmax(six_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for six modalities is {six_mod_accuracies[max_acc_index]:.2f} for {six_mod_combinations[max_acc_index]}")

In [ ]:
best_six = [MODALITY_INDICES[modality] for modality in six_mod_combinations[max_acc_index]]
print(best_six)
evaluate_fused_model(best_six)

## FIVE MODALITIES

In [ ]:
five_mod_accuracies, five_mod_combinations = train_combinations(num_modalites=5)

In [ ]:
max_acc_index = np.argmax(five_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for five modalities is {five_mod_accuracies[max_acc_index]:.2f} for {five_mod_combinations[max_acc_index]}")


In [ ]:
# best_five = ['EEG', 'EOG', 'EMG', 'GSR', 'RESP']

best_five =  [MODALITY_INDICES[modality] for modality in five_mod_combinations[max_acc_index]]
print(best_five)
evaluate_fused_model(best_five)

## FOUR MODALITIES

In [ ]:
four_mod_accuracies, four_mod_combinations = train_combinations(num_modalites=4)

In [ ]:
max_acc_index = np.argmax(four_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for four modalities is {four_mod_accuracies[max_acc_index]:.2f} for {four_mod_combinations[max_acc_index]}")


In [ ]:

best_four =  [MODALITY_INDICES[modality] for modality in four_mod_combinations[max_acc_index]]
print(best_four)
evaluate_fused_model(best_four)

## THREE MODALITIES

In [ ]:
three_mod_accuracies, three_mod_combinations = train_combinations(num_modalites=3)

In [ ]:
max_acc_index = np.argmax(three_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for three modalities is {three_mod_accuracies[max_acc_index]:.2f} for {three_mod_combinations[max_acc_index]}")


In [ ]:
# best_three = ['EOG', 'GSR', 'temp']

best_three = [MODALITY_INDICES[modality] for modality in three_mod_combinations[max_acc_index]]
print(best_three)
evaluate_fused_model(best_three)

## TWO MODALITIES

In [ ]:
two_mod_accuracies, two_mod_combinations = train_combinations(num_modalites=2)

In [ ]:
max_acc_index = np.argmax(two_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for two modalities is {two_mod_accuracies[max_acc_index]:.2f} for {two_mod_combinations[max_acc_index]}")


In [ ]:

best_two =  [MODALITY_INDICES[modality] for modality in two_mod_combinations[max_acc_index]]
print(best_two)
evaluate_fused_model(best_two)

## ONE MODALITY

In [ ]:
one_mod_accuracies, one_mod_combinations = train_combinations(num_modalites=1)

In [ ]:
max_acc_index = np.argmax(one_mod_accuracies) 
print(f"Highest accuracy from all possible combinations for one modalities is {one_mod_accuracies[max_acc_index]:.2f} for {one_mod_combinations[max_acc_index]}")
